In [1]:
# !pip install opencv-python scikit-learn scikit-image matplotlib spectral keras_tuner vis
# !pip install tensorflow numpy pandas
# !pip install spectral

In [2]:
import tensorflow as tf

In [3]:
from tensorflow import keras as keras
from keras import layers as layers
import numpy as np

In [4]:
import os, timeit
import cv2
from skimage.filters import threshold_otsu
import matplotlib.pyplot as plt
from math import inf as inf
import numpy as np

In [5]:
from spectral.io import envi as envi
from spectral import imshow

In [6]:
from sklearn.decomposition import IncrementalPCA

In [7]:
import keras_tuner as kt
import sys

In [8]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [9]:
from sys import platform
DATA_DIRECTORY = ""
SLASH = ""
if platform == "linux" or platform == "linux2":
    DATA_DIRECTORY = "/home/nitintyagi/wheat data/BULK/"
    SLASH = "/"
elif platform == "win32":
    DATA_DIRECTORY = "D:\mvl\wheat\data\BULK\\"
    SLASH="\\"

In [10]:
#Testing Constants
TESTING = False

#Constants
BAND_NUMBER = 60
FILLED_AREA_RATIO = 0.90
IMAGE_COUNT = int(1200/4)
NUM_VARIETIES = 4
NUM_OF_BANDS = 15
FIRST_BAND = 21
LAST_BAND = 149

IMAGE_WIDTH = 30
IMAGE_HEIGHT = 30
NUM_LAYERS_EACH_BLOCK = [1, 1]
NUM_CHAN_PER_BLOCK = [[64,128,128],[128,128,256]]

ACTIVATION_TYPE =  "relu"
BATCH_SIZE = 2*NUM_VARIETIES

LEARNING_RATE_BASE = 0.0001

In [11]:
def start_timer():
    print("Testing started")
    return timeit.default_timer()

def end_timer():
    return timeit.default_timer()

def show_time(tic,toc): 
    test_time = toc - tic
    print('Testing time (s) = ' + str(test_time) + '\n')

In [12]:
def exactPathHDR(variety,file):
    return DATA_DIRECTORY+variety+SLASH+file+".bil.hdr"

def exactPathBIL(variety,file):
    return DATA_DIRECTORY+variety+SLASH+file+".bil"

In [13]:
def getROI(img,band_number):
    img_band = img.read_band(band_number)
    threshold = threshold_otsu(img_band)
    roi=[]
    for x in range(img_band.shape[0]):
        a=[]
        for y in range(img_band.shape[1]):
            if img_band[x][y]>threshold:
                a.append(1)
            else:
                a.append(0)
        roi.append(a)
    return roi

In [14]:
#Returns range for x and y from where we have to crop images
def getRangeXandY(img,band_number):
    img_band = img.read_band(band_number)
    roi = getROI(img,band_number)
    xmin = inf
    xmax = 0
    ymin = inf
    ymax = 0
    for x in range(img_band.shape[0]):
        for y in range(img_band.shape[1]):
            if roi[x][y]==1:
                if x<xmin:
                    xmin=x
                if x>xmax:
                    xmax=x
                if y<ymin:
                    ymin=y
                if y>ymax:
                    ymax=y
    return xmin, xmax, ymin, ymax

In [15]:
def getCroppedImage(img,band_number):
    xmin, xmax, ymin, ymax = getRangeXandY(img,band_number)
    new_img = img[xmin:xmax, ymin:ymax, :]
    return new_img    

In [16]:
def getCroppedROI(img,band_number):
    xmin, xmax, ymin, ymax = getRangeXandY(img,band_number)
    roi = np.array(getROI(img,band_number))
    roi = roi[xmin:xmax, ymin:ymax]
    return roi   

In [17]:
def getUsefulImage(img,band_number):
    crop_img = getCroppedImage(img,band_number)
    crop_roi = getCroppedROI(img,band_number)
    for x in range(crop_img.shape[2]):
        band = crop_img[:,:,x]
        crop_img[:,:,x] = band*crop_roi
    return crop_img

In [18]:
data_augmentation = keras.Sequential([
    layers.RandomCrop(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
    layers.RandomRotation(factor=(-0.1, 0.1)),
    layers.RandomZoom(height_factor=(-0.1, 0.1), width_factor=(-0.1,0.1)),
    layers.RandomFlip(mode="horizontal_and_vertical", seed=None)
])

def getAugumentedImage(img,band_number):
    new_img = getUsefulImage(img,band_number)
    augmented_image = data_augmentation(new_img) 
    return augmented_image

def checkAugumentedImage(augmented_image):
    aug_band = augmented_image[:,:,0]
    filled_area_ratio = (np.count_nonzero(aug_band))/(aug_band.shape[0]*aug_band.shape[1])
    if filled_area_ratio > FILLED_AREA_RATIO :
        return True
    else:
        return False

In [19]:
## Dimensional Reduction Method
def DL_Method(HSI, numComponents = NUM_OF_BANDS):
    RHSI = np.reshape(HSI, (-1, HSI.shape[2]))
    n_batches = 10
    inc_pca = IncrementalPCA(n_components=numComponents)
    for X_batch in np.array_split(RHSI, n_batches):
        inc_pca.partial_fit(X_batch)
    X_ipca = inc_pca.transform(RHSI)
    RHSI = np.reshape(X_ipca, (HSI.shape[0],HSI.shape[1], numComponents))
    return RHSI

In [20]:
# List for All varieties
VARIETIES = []
VARIETIES_CODE = {}

for name in os.listdir(DATA_DIRECTORY):
    if (name.endswith(".hdr") or name.endswith(".bil")):
        continue
    VARIETIES_CODE[name] = len(VARIETIES)
    VARIETIES.append(name)
    if len(VARIETIES)==NUM_VARIETIES:
        break

In [21]:
#List for all file names in varities
FILES = []
MAX_FILE_NUM = 4
for x in range(1,MAX_FILE_NUM+1):
    FILES.append("B_"+str(x))

In [22]:
#List of all images
images = []
images_label = []
for v in VARIETIES:
    for f in FILES:
        try:
            img = envi.open(exactPathHDR(v,f),exactPathBIL(v,f))
            images.append(img)
            images_label.append(v)
        except:
            pass

In [23]:
train_dataset = []
train_dataset_label = []
test_dataset = []
test_dataset_label = []

In [24]:
i = 0

In [25]:
tic = start_timer()
for index, img in enumerate(images):
    count = 0
    label = images_label[index]
    while count<IMAGE_COUNT:
        aug_img = getAugumentedImage(img,BAND_NUMBER)
        
        if checkAugumentedImage(aug_img):
            aug_img = DL_Method(aug_img[:,:,FIRST_BAND:LAST_BAND+1])
            if count%5 == 0:
                test_dataset.append(aug_img)
                test_dataset_label.append(label)
            else:
                train_dataset.append(aug_img)
                train_dataset_label.append(label)
            count+=1 
        
        i+=1
        print(i,count)    
    
    if TESTING:
        break
        
toc = end_timer()
show_time(tic,toc)

Testing started
1 1
2 2
3 3
4 3
5 4
6 4
7 4
8 5
9 5
10 6
11 6
12 7
13 7
14 7
15 7
16 7
17 8
18 8
19 9
20 10
21 10
22 10
23 10
24 11
25 12
26 13
27 13
28 14
29 15
30 16
31 16
32 16
33 16
34 16
35 17
36 17
37 18
38 18
39 18
40 18
41 18
42 19
43 20
44 21
45 21
46 21
47 21
48 22
49 22
50 23
51 23
52 23
53 23
54 24
55 25
56 25
57 25
58 25
59 26
60 26
61 26
62 26
63 27
64 27
65 27
66 28
67 28
68 28
69 29
70 29
71 30
72 31
73 32
74 33
75 34
76 35
77 35
78 35
79 35
80 36
81 37
82 37
83 37
84 38
85 38
86 38
87 39
88 39
89 39
90 40
91 41
92 42
93 43
94 43
95 44
96 44
97 45
98 45
99 45
100 45
101 46
102 47
103 47
104 48
105 48
106 49
107 50
108 50
109 51
110 52
111 53
112 53
113 53
114 54
115 54
116 55
117 56
118 56
119 57
120 57
121 57
122 57
123 58
124 58
125 58
126 58
127 58
128 58
129 59
130 59
131 59
132 59
133 59
134 60
135 61
136 62
137 62
138 63
139 63
140 63
141 63
142 63
143 64
144 65
145 65
146 65
147 66
148 66
149 67
150 67
151 68
152 69
153 69
154 70
155 70
156 71
157 71
158 71
159 7

1097 109
1098 109
1099 109
1100 109
1101 109
1102 109
1103 109
1104 110
1105 110
1106 110
1107 110
1108 111
1109 111
1110 111
1111 112
1112 113
1113 113
1114 113
1115 114
1116 114
1117 114
1118 115
1119 115
1120 115
1121 115
1122 116
1123 117
1124 118
1125 119
1126 119
1127 119
1128 119
1129 119
1130 119
1131 119
1132 119
1133 120
1134 121
1135 121
1136 121
1137 122
1138 123
1139 124
1140 125
1141 125
1142 126
1143 126
1144 126
1145 126
1146 126
1147 127
1148 128
1149 129
1150 130
1151 130
1152 131
1153 131
1154 132
1155 132
1156 133
1157 133
1158 133
1159 133
1160 133
1161 134
1162 135
1163 135
1164 135
1165 135
1166 135
1167 135
1168 136
1169 137
1170 137
1171 137
1172 137
1173 137
1174 137
1175 137
1176 137
1177 138
1178 139
1179 139
1180 140
1181 140
1182 140
1183 141
1184 142
1185 143
1186 143
1187 144
1188 144
1189 144
1190 144
1191 144
1192 144
1193 144
1194 145
1195 145
1196 145
1197 145
1198 145
1199 145
1200 145
1201 145
1202 145
1203 146
1204 146
1205 146
1206 146
1207 146
1

2042 148
2043 148
2044 148
2045 148
2046 148
2047 148
2048 149
2049 149
2050 149
2051 149
2052 150
2053 151
2054 152
2055 152
2056 152
2057 152
2058 153
2059 154
2060 154
2061 154
2062 154
2063 154
2064 154
2065 155
2066 156
2067 156
2068 156
2069 157
2070 157
2071 157
2072 158
2073 159
2074 159
2075 159
2076 159
2077 160
2078 161
2079 161
2080 161
2081 161
2082 161
2083 161
2084 162
2085 163
2086 164
2087 164
2088 164
2089 165
2090 165
2091 165
2092 165
2093 166
2094 166
2095 166
2096 167
2097 167
2098 167
2099 167
2100 167
2101 167
2102 167
2103 168
2104 168
2105 169
2106 169
2107 170
2108 170
2109 171
2110 172
2111 173
2112 173
2113 173
2114 173
2115 173
2116 173
2117 174
2118 174
2119 174
2120 175
2121 175
2122 175
2123 175
2124 176
2125 176
2126 177
2127 177
2128 178
2129 178
2130 178
2131 178
2132 179
2133 179
2134 180
2135 181
2136 181
2137 181
2138 181
2139 182
2140 182
2141 182
2142 182
2143 182
2144 183
2145 184
2146 184
2147 185
2148 186
2149 186
2150 186
2151 186
2152 186
2

2976 4
2977 4
2978 4
2979 4
2980 4
2981 4
2982 5
2983 6
2984 7
2985 7
2986 8
2987 8
2988 9
2989 9
2990 9
2991 10
2992 11
2993 11
2994 11
2995 12
2996 12
2997 12
2998 12
2999 13
3000 14
3001 14
3002 15
3003 16
3004 17
3005 18
3006 18
3007 19
3008 20
3009 20
3010 20
3011 21
3012 21
3013 21
3014 21
3015 22
3016 22
3017 23
3018 24
3019 24
3020 24
3021 24
3022 24
3023 25
3024 26
3025 27
3026 28
3027 28
3028 28
3029 29
3030 30
3031 31
3032 31
3033 32
3034 33
3035 33
3036 33
3037 34
3038 34
3039 35
3040 35
3041 36
3042 37
3043 37
3044 37
3045 37
3046 37
3047 38
3048 39
3049 40
3050 40
3051 40
3052 40
3053 41
3054 42
3055 42
3056 42
3057 43
3058 43
3059 43
3060 44
3061 45
3062 45
3063 45
3064 45
3065 45
3066 46
3067 46
3068 46
3069 47
3070 48
3071 48
3072 48
3073 49
3074 50
3075 51
3076 51
3077 51
3078 51
3079 52
3080 52
3081 52
3082 53
3083 53
3084 53
3085 53
3086 53
3087 53
3088 54
3089 54
3090 54
3091 54
3092 55
3093 56
3094 56
3095 57
3096 57
3097 58
3098 59
3099 59
3100 59
3101 60
3102 60

3941 84
3942 84
3943 85
3944 85
3945 85
3946 85
3947 85
3948 85
3949 85
3950 85
3951 86
3952 87
3953 88
3954 88
3955 88
3956 88
3957 88
3958 89
3959 89
3960 90
3961 91
3962 92
3963 92
3964 93
3965 93
3966 94
3967 94
3968 94
3969 94
3970 95
3971 95
3972 95
3973 95
3974 95
3975 95
3976 96
3977 96
3978 96
3979 96
3980 96
3981 97
3982 98
3983 98
3984 99
3985 100
3986 101
3987 101
3988 102
3989 103
3990 103
3991 104
3992 104
3993 104
3994 104
3995 105
3996 106
3997 107
3998 108
3999 109
4000 109
4001 110
4002 110
4003 111
4004 111
4005 111
4006 111
4007 112
4008 112
4009 112
4010 112
4011 112
4012 113
4013 113
4014 113
4015 114
4016 114
4017 115
4018 116
4019 116
4020 116
4021 116
4022 117
4023 117
4024 117
4025 118
4026 118
4027 118
4028 118
4029 119
4030 119
4031 119
4032 120
4033 120
4034 120
4035 121
4036 121
4037 121
4038 121
4039 121
4040 121
4041 121
4042 121
4043 122
4044 122
4045 122
4046 123
4047 123
4048 124
4049 125
4050 125
4051 126
4052 127
4053 127
4054 127
4055 127
4056 128


...

8662 2
8663 2
8664 2
8665 3
8666 4
8667 5
8668 6
8669 6
8670 6
8671 6
8672 6
8673 7
8674 7
8675 8
8676 9
8677 9
8678 10
8679 10
8680 10
8681 10
8682 10
8683 10
8684 11
8685 11
8686 12
8687 12
8688 12
8689 13
8690 14
8691 14
8692 14
8693 15
8694 16
8695 17
8696 18
8697 18
8698 19
8699 20
8700 20
8701 21
8702 22
8703 22
8704 22
8705 22
8706 22
8707 23
8708 23
8709 24
8710 25
8711 25
8712 26
8713 27
8714 28
8715 29
8716 29
8717 29
8718 29
8719 30
8720 30
8721 31
8722 31
8723 31
8724 32
8725 32
8726 33
8727 34
8728 34
8729 34
8730 35
8731 36
8732 36
8733 36
8734 37
8735 38
8736 38
8737 39
8738 39
8739 40
8740 40
8741 41
8742 42
8743 42
8744 43
8745 44
8746 45
8747 46
8748 47
8749 48
8750 48
8751 48
8752 49
8753 49
8754 50
8755 50
8756 50
8757 51
8758 52
8759 52
8760 53
8761 54
8762 55
8763 55
8764 56
8765 56
8766 57
8767 57
8768 57
8769 58
8770 58
8771 59
8772 60
8773 60
8774 61
8775 62
8776 63
8777 63
8778 63
8779 63
8780 63
8781 64
8782 64
8783 65
8784 66
8785 67
8786 67
8787 67
8788 68


9633 101
9634 101
9635 101
9636 101
9637 101
9638 101
9639 101
9640 102
9641 103
9642 103
9643 103
9644 103
9645 104
9646 104
9647 104
9648 104
9649 104
9650 105
9651 105
9652 105
9653 105
9654 105
9655 106
9656 106
9657 107
9658 108
9659 109
9660 110
9661 110
9662 111
9663 112
9664 113
9665 114
9666 114
9667 114
9668 114
9669 114
9670 114
9671 115
9672 115
9673 115
9674 115
9675 115
9676 115
9677 115
9678 116
9679 116
9680 116
9681 117
9682 117
9683 117
9684 118
9685 119
9686 119
9687 119
9688 119
9689 119
9690 120
9691 120
9692 120
9693 120
9694 120
9695 121
9696 121
9697 121
9698 122
9699 122
9700 123
9701 123
9702 123
9703 123
9704 123
9705 123
9706 123
9707 124
9708 124
9709 125
9710 126
9711 126
9712 126
9713 126
9714 127
9715 127
9716 127
9717 127
9718 127
9719 127
9720 127
9721 127
9722 128
9723 128
9724 129
9725 130
9726 131
9727 131
9728 132
9729 132
9730 132
9731 132
9732 132
9733 132
9734 133
9735 134
9736 135
9737 135
9738 135
9739 135
9740 136
9741 137
9742 138
9743 139
9

10517 128
10518 129
10519 129
10520 129
10521 129
10522 129
10523 129
10524 129
10525 130
10526 130
10527 130
10528 131
10529 131
10530 132
10531 132
10532 133
10533 133
10534 133
10535 134
10536 134
10537 134
10538 134
10539 134
10540 135
10541 135
10542 135
10543 135
10544 135
10545 135
10546 135
10547 135
10548 136
10549 136
10550 136
10551 136
10552 136
10553 136
10554 137
10555 138
10556 138
10557 138
10558 138
10559 138
10560 138
10561 138
10562 139
10563 140
10564 141
10565 142
10566 142
10567 143
10568 143
10569 144
10570 144
10571 144
10572 144
10573 144
10574 145
10575 146
10576 147
10577 147
10578 148
10579 148
10580 149
10581 149
10582 150
10583 151
10584 151
10585 151
10586 151
10587 151
10588 152
10589 153
10590 153
10591 153
10592 153
10593 154
10594 155
10595 155
10596 156
10597 157
10598 157
10599 157
10600 157
10601 157
10602 157
10603 157
10604 157
10605 157
10606 158
10607 158
10608 158
10609 158
10610 159
10611 160
10612 161
10613 161
10614 161
10615 161
10616 161


11367 137
11368 137
11369 137
11370 137
11371 137
11372 138
11373 138
11374 139
11375 139
11376 139
11377 140
11378 140
11379 141
11380 141
11381 142
11382 143
11383 143
11384 143
11385 144
11386 144
11387 144
11388 144
11389 144
11390 144
11391 144
11392 144
11393 144
11394 144
11395 144
11396 144
11397 145
11398 145
11399 146
11400 146
11401 146
11402 146
11403 146
11404 146
11405 146
11406 147
11407 148
11408 148
11409 148
11410 149
11411 150
11412 150
11413 151
11414 151
11415 151
11416 151
11417 152
11418 152
11419 152
11420 152
11421 153
11422 153
11423 153
11424 153
11425 153
11426 154
11427 154
11428 154
11429 154
11430 154
11431 155
11432 155
11433 155
11434 155
11435 156
11436 157
11437 157
11438 157
11439 157
11440 157
11441 157
11442 157
11443 157
11444 158
11445 159
11446 160
11447 161
11448 162
11449 162
11450 162
11451 162
11452 162
11453 163
11454 164
11455 165
11456 165
11457 165
11458 165
11459 166
11460 166
11461 167
11462 167
11463 167
11464 167
11465 167
11466 167


12204 244
12205 245
12206 246
12207 246
12208 247
12209 248
12210 249
12211 249
12212 249
12213 249
12214 250
12215 251
12216 252
12217 253
12218 254
12219 255
12220 255
12221 256
12222 256
12223 257
12224 257
12225 257
12226 257
12227 258
12228 258
12229 259
12230 260
12231 261
12232 261
12233 262
12234 262
12235 263
12236 263
12237 263
12238 264
12239 264
12240 265
12241 266
12242 267
12243 267
12244 268
12245 269
12246 269
12247 269
12248 269
12249 269
12250 269
12251 270
12252 271
12253 271
12254 272
12255 272
12256 272
12257 273
12258 274
12259 274
12260 275
12261 275
12262 276
12263 277
12264 277
12265 278
12266 279
12267 280
12268 281
12269 281
12270 282
12271 283
12272 284
12273 285
12274 286
12275 287
12276 288
12277 288
12278 289
12279 289
12280 290
12281 291
12282 292
12283 293
12284 294
12285 295
12286 296
12287 297
12288 298
12289 299
12290 299
12291 300
Testing time (s) = 6406.107069799999



In [26]:
train_dataset = np.array(train_dataset)
train_dataset_label = np.array([VARIETIES_CODE[label] for label in train_dataset_label])
test_dataset = np.array(test_dataset)
test_dataset_label = np.array([VARIETIES_CODE[label] for label in test_dataset_label])

In [27]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import pyplot as plt
# import math, sys, pdb, os
import keras
import keras.backend as K
import tensorflow as tf
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, BatchNormalization, Add, Conv2DTranspose, Flatten, Dense, Conv1D, AveragePooling2D, LeakyReLU, PReLU, GlobalAveragePooling2D
from keras.layers.core import Dropout
from keras.layers import concatenate
from keras.models import Model

import os, pdb, timeit
import numpy as np
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import matplotlib.cm as cm
import cv2
from keras import activations
import vis
# from vis.visualization import visualize_saliency, overlay
# from vis.utils import utils

In [28]:
def normalizeDataWholeSeed(data,normalization_type='max'):
    
    if normalization_type == 'max':
        for idx in range(data.shape[0]):
            data[idx,:,:,:] = data[idx,:,:,:]/np.max(abs(data[idx,:,:,:]))
            
    elif normalization_type == 'l2norm':
        from numpy import linalg as LA
        for idx in range(data.shape[0]):
            data[idx,:,:,:] = data[idx,:,:,:]/LA.norm(data[idx,:,:,:]) # L2-norm by default        
        
    return data

In [29]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.clim(0,sum(cm[0,:]))
    plt.xlabel('Predicted label')

In [30]:
def top_K_classification_accuracy(y_predicted, y_true, K=1):

    num_samples = y_predicted.shape[0]
    num_classes = y_predicted.shape[1]

    if K > num_classes:
        sys.exit(1)

    temp = np.zeros((num_samples,))

    for idx in range(num_samples):
        curr_predicted = np.argsort(y_predicted[idx,:])
        curr_predicted = curr_predicted[::-1] # descending

        if y_true[idx] in curr_predicted[:K]:
            temp[idx] = 1

    return 100.0 * np.sum(temp)/num_samples

In [31]:
def single_conv_block(x, kernel_size=3, activation_type='relu', num_filters=128,use_bias=False):
    # Batch norm
    x = BatchNormalization()(x)
    x = Conv2D(num_filters, kernel_size, activation=None, use_bias=use_bias, padding='same',
               kernel_initializer='truncated_normal')(x)

    # Activation
    x = Activation(activation_type)(x)
    return x

In [32]:
def conv2D_ResNet(x, kernel_size, activation_type, dropout_rate, num_filters_first_conv1D=[32,64,128]):

    x = single_conv_block(x,1,activation_type,num_filters_first_conv1D[0])

    x = single_conv_block(x,kernel_size,activation_type,num_filters_first_conv1D[1],use_bias=True)

    x = single_conv_block(x,1,activation_type,num_filters_first_conv1D[2])

    # Dropout
    return Dropout(dropout_rate)(x)

In [33]:
def createBlock_ResNet2D(x, num_layers, kernel_size, activation_type, dropout_rate, num_filters_first_conv1D):

    for idx_layer in range(num_layers):
        x = conv2D_ResNet(x, kernel_size, activation_type, dropout_rate, num_filters_first_conv1D)

    return x

In [34]:
def ResNet2D_classifier(data_num_rows, data_num_cols, num_classes, kernel_size=3, num_layers_each_block=[],
                        num_chan_per_block=[], activation_type='relu', dropout_rate=0.0, num_input_chans=1, num_nodes_fc=64):

    input_data = Input(shape=(data_num_rows, data_num_cols, num_input_chans))

    # Input layer: Conv2D -> activation
    x = Conv2D(num_chan_per_block[0][0], kernel_size, activation=None, use_bias=True, padding='same',
               kernel_initializer='truncated_normal')(input_data)

    # Activation
    x = Activation(activation_type)(x)


    #  Blocks & Downsampling Layers
    for idx_block in range(len(num_layers_each_block)):
        x = createBlock_ResNet2D(x, num_layers_each_block[idx_block], kernel_size, activation_type, dropout_rate,
                                 num_chan_per_block[idx_block])

        x = BatchNormalization()(x)

        if idx_block != len(num_layers_each_block)-1:
            x = Conv2D(num_chan_per_block[idx_block][1]*2, kernel_size, strides = 2, activation=None, use_bias=True, padding='valid',
                   kernel_initializer='truncated_normal')(x)
        else:
            x = GlobalAveragePooling2D()(x)

        x = Dropout(dropout_rate)(x)

    # Output layer
    x = BatchNormalization()(x)
    
    x = Flatten()(x)

    x = Dense(units=num_nodes_fc, activation=None, kernel_initializer='truncated_normal')(x)
#     x = Dense(units=32, activation=None, kernel_initializer='truncated_normal')(x)

    # Activation
    x = Activation(activation_type)(x)

    x = BatchNormalization()(x)
    output_data = Dense(units=num_classes, activation='softmax', kernel_initializer='truncated_normal')(x)

    return Model(inputs=input_data, outputs=output_data)

In [35]:
def evaluate(model,dataset,dataset_label,normalization_type):
    print("--------------Make Predictions--------------")    
    x = np.array(dataset)
    labels = np.array(dataset_label)
    
    # Normalize the data
    x = normalizeDataWholeSeed(x,normalization_type=normalization_type)
    
    num = x.shape[0]

    tic = start_timer()
    labels_predicted = model.predict(x)
    toc = end_timer()
    show_time(tic,toc)
    
    print(labels_predicted)
    print("--------")
    # Classification accuracy
    labels_integer_format = labels
    labels_predicted_integer_format = np.argmax(labels_predicted, axis=1)

    acc_top2 = top_K_classification_accuracy(labels_predicted, labels_integer_format, K=2)
    acc_top1 = top_K_classification_accuracy(labels_predicted, labels_integer_format, K=1)
    
    # Confusion matrices
    confusion_matrix_results = confusion_matrix(labels_integer_format, labels_predicted_integer_format)
    print("Confusion matrix = ")
    print(confusion_matrix_results)
    print("------------------------------------------------")

In [36]:
def predict(model,normalization_type):
    evaluate(model,train_dataset,train_dataset_label,normalization_type)
    
    evaluate(model,test_dataset,test_dataset_label,normalization_type)
    


In [37]:
def createAndTrainResNetB():
    
    learning_rate_base = LEARNING_RATE_BASE
    kernel_size = 3
    dropout_rate = 0.15 
    activation_type = ACTIVATION_TYPE
    num_nodes_fc = 256
    wheat_types =  VARIETIES
    normalization_type = 'max'
    num_layers_each_block = NUM_LAYERS_EACH_BLOCK
    num_chan_per_block = NUM_CHAN_PER_BLOCK
    N_classes = len(wheat_types)
    
    ############ Load data ############
    print("--------------Load Data--------------")

    # Load training data and their corresponding labels
    x_training = np.array(train_dataset)
    labels_training = np.array(train_dataset_label)
    
    # Normalize the data
    x_training = normalizeDataWholeSeed(x_training,normalization_type=normalization_type)
    
    # Extract some information
    num_training = x_training.shape[0]
    N_spatial = x_training.shape[1:3]
    N_bands = x_training.shape[3]
    
    print("--------------Done--------------")
    
    ############ Create a model ############
    print("--------------Create a model--------------")
    
    # Generate a model
    model = ResNet2D_classifier(data_num_rows = N_spatial[0], 
                                data_num_cols = N_spatial[1], 
                                num_classes = N_classes,
                                kernel_size = kernel_size, 
                                num_layers_each_block = num_layers_each_block,
                                num_chan_per_block = num_chan_per_block, 
                                activation_type = activation_type,
                                dropout_rate = dropout_rate, 
                                num_input_chans = N_bands, 
                                num_nodes_fc = num_nodes_fc)

    # Compile the model
    adam_opt = Adam(learning_rate=LEARNING_RATE_BASE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=adam_opt, metrics=['accuracy'])
    print("---------Completed---------")
    return model

In [38]:
model = createAndTrainResNetB()

--------------Load Data--------------
--------------Done--------------
--------------Create a model--------------
---------Completed---------


In [39]:
for x in range(4):
    print("From epochs: ",20*x+1," to ",20+20*x)
    tic = start_timer()
    print(model.fit(x=train_dataset,y=train_dataset_label,batch_size=BATCH_SIZE, epochs=20, initial_epoch = 0, verbose=2,validation_split=0.40,shuffle=True))
    toc = end_timer()
    show_time(tic,toc)
    print("for testing")
    print(model.evaluate(test_dataset,test_dataset_label))
    print("for training")
    print(model.evaluate(train_dataset,train_dataset_label))

From epochs:  1  to  20
Testing started
Epoch 1/20
288/288 - 63s - loss: 1.0889 - accuracy: 0.5634 - val_loss: 2.4299 - val_accuracy: 0.0000e+00 - 63s/epoch - 220ms/step
Epoch 2/20
288/288 - 60s - loss: 0.8589 - accuracy: 0.6975 - val_loss: 1.8338 - val_accuracy: 0.0807 - 60s/epoch - 207ms/step
Epoch 3/20
288/288 - 63s - loss: 0.7806 - accuracy: 0.7361 - val_loss: 1.6324 - val_accuracy: 0.2018 - 63s/epoch - 217ms/step
Epoch 4/20
288/288 - 56s - loss: 0.7189 - accuracy: 0.7613 - val_loss: 1.7149 - val_accuracy: 0.2103 - 56s/epoch - 194ms/step
Epoch 5/20
288/288 - 55s - loss: 0.7149 - accuracy: 0.7504 - val_loss: 1.8364 - val_accuracy: 0.2207 - 55s/epoch - 191ms/step
Epoch 6/20
288/288 - 56s - loss: 0.6886 - accuracy: 0.7700 - val_loss: 1.9056 - val_accuracy: 0.2246 - 56s/epoch - 193ms/step
Epoch 7/20
288/288 - 56s - loss: 0.6423 - accuracy: 0.7904 - val_loss: 1.9254 - val_accuracy: 0.2350 - 56s/epoch - 193ms/step
Epoch 8/20
288/288 - 57s - loss: 0.6309 - accuracy: 0.7951 - val_loss: 1.9

Epoch 19/20
288/288 - 58s - loss: 0.4927 - accuracy: 0.8381 - val_loss: 2.2583 - val_accuracy: 0.2630 - 58s/epoch - 201ms/step
Epoch 20/20
288/288 - 58s - loss: 0.5032 - accuracy: 0.8316 - val_loss: 2.2503 - val_accuracy: 0.2604 - 58s/epoch - 202ms/step
Testing time (s) = 1154.4628795999997

for testing
30/30 [==============================] - 4s 144ms/step - loss: 1.2148 - accuracy: 0.5792
[1.2147547006607056, 0.5791666507720947]
for training
120/120 [==============================] - 17s 146ms/step - loss: 1.1180 - accuracy: 0.6310
[1.118018388748169, 0.6309896111488342]
From epochs:  61  to  80
Testing started
Epoch 1/20
288/288 - 58s - loss: 0.4971 - accuracy: 0.8355 - val_loss: 2.2656 - val_accuracy: 0.2546 - 58s/epoch - 201ms/step
Epoch 2/20
288/288 - 59s - loss: 0.4708 - accuracy: 0.8520 - val_loss: 2.2728 - val_accuracy: 0.2611 - 59s/epoch - 204ms/step
Epoch 3/20
288/288 - 58s - loss: 0.5050 - accuracy: 0.8329 - val_loss: 2.2806 - val_accuracy: 0.2591 - 58s/epoch - 202ms/step
E